In [1]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))
sys.path.append(project_path)
print(f"Project Path: {project_path}")

Project Path: c:\Users\dalej\Documents\_Coding\DragonRegen


## Examine Kafka Consumer Group IDs
Look at the current committed offsets of each kafka consumer group_id

In [11]:
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
from kafka import KafkaConsumer
from kafka import TopicPartition

# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

# List all consumer groups
consumer_groups = admin_client.list_consumer_groups()
group_ids = [group[0] for group in consumer_groups]  # Extract group IDs from tuples
print(f"Available consumer groups: {group_ids}")

# Create a consumer to get end offsets
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'])

# Get offsets for each consumer group
for group_id in group_ids:
    # Get offsets for all topics this consumer group is consuming
    offsets = admin_client.list_consumer_group_offsets(group_id)
    
    print(f"\nGroup: {group_id}")
    for tp, offset_info in offsets.items():
        print(f"  Topic: {tp.topic}, Partition: {tp.partition}, Offset: {offset_info.offset}")
        
        # Optional: Get lag by comparing with end offsets
        consumer_position = offset_info.offset
        
        # Create a TopicPartition object
        topic_partition = TopicPartition(tp.topic, tp.partition)
        
        # Get the end offset for this topic-partition using the consumer
        consumer.assign([topic_partition])
        end_offset = consumer.end_offsets([topic_partition])[topic_partition]
        
        # Calculate lag
        lag = end_offset - consumer_position
        print(f"    end_offset: {end_offset}")
        print(f"    consumer_position: {consumer_position}")
        print(f"    Lag: {lag}")

Available consumer groups: ['action-agent', 'default']

Group: action-agent
  Topic: ai_tasks, Partition: 0, Offset: 8
    end_offset: 8
    consumer_position: 8
    Lag: 0

Group: default
  Topic: ai_tasks_completed, Partition: 0, Offset: 3
    end_offset: 3
    consumer_position: 3
    Lag: 0


In [20]:
# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

admin_client.delete_topics(['ai_tasks_completed'])
print("Topic 'ai_tasks_log' deleted successfully.")

Topic 'ai_tasks_log' deleted successfully.


## Look at all topics

In [8]:
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
from kafka import KafkaConsumer
from kafka import TopicPartition

# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

# List all consumer groups
consumer_groups = admin_client.list_consumer_groups()
print(f"Available consumer groups: {consumer_groups}")

# List all topics
topics = admin_client.list_topics()
print(f"Available topics: {topics}")

# Find the length of ai_tasks_completed topic

topic_name = 'ai_tasks_completed'
if topic_name in topics:
    print(admin_client.describe_topics([topic_name])[0])
    # max offset of this topic
    topic_partition = TopicPartition(topic_name, 0)
    consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'])
    consumer.assign([topic_partition])
    end_offset = consumer.end_offsets([topic_partition])[topic_partition]
    print(f"End offset for topic '{topic_name}': {end_offset}")

Available consumer groups: [('action-agent', 'consumer')]
Available topics: ['ai_tasks_completed', 'quickstart-events', 'ai_requests_log', 'ai_tasks_log', 'ai_tasks', '__consumer_offsets']
{'error_code': 0, 'topic': 'ai_tasks_completed', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 0, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}]}
End offset for topic 'ai_tasks_completed': 2


In [14]:
from kafka.admin import KafkaAdminClient, NewTopic
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

new_topic = NewTopic(
    name='many_tasks',
    num_partitions=5,
    replication_factor=1
)
admin_client.create_topics([new_topic])


CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='many_tasks', error_code=0, error_message=None)])

In [22]:
from kafka.admin import KafkaAdminClient, ConfigResource, ConfigResourceType
from kafka import KafkaConsumer

# Initialize the admin client
admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])

# Method 1: Get all topics and their partitions using describe_topics() 
topic_metadata = admin_client.describe_topics()
for topic_info in topic_metadata:
    print(topic_info)
    partition_count = len(topic_info['partitions'])
    print(f"Topic: {topic_name}, Partitions: {partition_count}")

{'error_code': 0, 'topic': 'ai_tasks_completed', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 1, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 3, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 2, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 0, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}, {'error_code': 0, 'partition': 4, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}]}
Topic: ai_tasks_completed, Partitions: 5
{'error_code': 0, 'topic': 'quickstart-events', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 0, 'leader': 1, 'replicas': [1], 'isr': [1], 'offline_replicas': []}]}
Topic: ai_tasks_completed, Partitions: 1
{'error_code': 0, 'topic': 'ai_requests_log', 'is_internal': False, 'partitions': [{'error_code': 0, 'partition': 1, 'leader': 1, 'replicas